In [3]:
import pandas as pd
#import datadotworld as ddw
import log_progress # https://github.com/alexanderkuk/log-progress
import requests
from dbfread import DBF
import os
from zipfile import ZipFile
from ftplib import FTP
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [4]:
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#pd.set_option('max_colwidth', 70)

In [5]:
root = '/Users/ntefft'
os.mkdir('%s/Desktop/FARS' % root)
os.chdir('%s/Desktop/FARS' % root)

In [6]:
ftp = FTP('ftp.nhtsa.dot.gov')
ftp.login()

'230 User logged in.'

In [7]:

for year in log_progress.log_progress(range(1982,2018)):
    if os.getcwd() != '%s/Desktop/FARS' % root:
        os.chdir('%s/Desktop/FARS' % root)
    filename = "f_%d.zip" % year
    if year < 2013:
        ftp_path = '/FARS/%s/Auxiliary_FARS_Files/' % year
        source_name = 'RETR Auxiliary_FARS_DBF_%s.ZIP' % year
    elif (year == 2013) | (year == 2014):
        ftp_path = '/fars/%s/National/' % year
        source_name = 'RETR FARS%sNationalDBF.zip' % year
    elif year > 2014:
        ftp_path = '/fars/%s/National/' % year
        source_name = 'RETR FARS%sNationalCSV.zip' % year
    else:
        print("problem", year)
    
    
    zfile = ZipFile(filename, 'w')
    zfile.close()
    
    ftp.cwd(ftp_path)
    
    ftp.retrbinary(source_name, open('%s/Desktop/FARS/%s' % (root, filename), mode='ab').write)
    ZipFile('f_%d.zip' % year, 'r').extractall('%s/Desktop/FARS/f_%s' % (root, year))
    
    if year % 5 == 0:
        print(year)
    

1985
1990
1995
2000
2005
2010
2015


In [6]:
for i in os.listdir():
    if i.endswith('.zip'):
        os.remove(i)

file_names = os.listdir()
file_names.sort()

dirs = {i: os.listdir(i) for i in file_names[1:]}
f_years = []
for i, j in dirs.items():
    if j[0] == 'FARS':
        f_years.append(i)
        dirs[i] = os.listdir('%s/FARS' % i)
        dirs[i] = ["%s_%s" % (i, k) for k in dirs[i]] 
    else:
        dirs[i] = ["%s_%s" % (i, k) for k in j]

In [7]:
keys = ['f_1982', 'f_1983', 'f_1984', 'f_1985', 'f_1986', 'f_1987', 
        'f_1988', 'f_1989', 'f_1990', 'f_1991', 'f_1992', 'f_1993', 
        'f_1994', 'f_1995', 'f_1996', 'f_1997', 'f_1998', 'f_1999', 
        'f_2000', 'f_2001', 'f_2002', 'f_2003', 'f_2004', 'f_2005', 
        'f_2006', 'f_2007', 'f_2008', 'f_2009', 'f_2010', 'f_2011', 
        'f_2012', 'f_2013', 'f_2014']
names = ['VEH_AUX.dbf', 'PER_AUX.dbf', 'ACC_AUX.dbf']

for name in log_progress.log_progress(names):
    count = 0
    for k in log_progress.log_progress(keys):
        if k in f_years:
            table = DBF('%s/FARS/%s' % (k,name), load=True)
        else:
            table = DBF('%s/%s' % (k,name), load=True)
        table.load()
        
        if count == 0:
            if name == 'VEH_AUX.dbf':
                VEH_AUX = pd.DataFrame(table.records)
            elif name == 'PER_AUX.dbf':
                PER_AUX = pd.DataFrame(table.records)
            elif name == 'ACC_AUX.dbf':
                ACC_AUX = pd.DataFrame(table.records)
        else:
            if name == 'VEH_AUX.dbf':
                VEH_AUX = pd.concat([VEH_AUX, pd.DataFrame(table.records)])
            elif name == 'PER_AUX.dbf':
                PER_AUX = pd.concat([PER_AUX, pd.DataFrame(table.records)])
            elif name == 'ACC_AUX.dbf':
                ACC_AUX = pd.concat([ACC_AUX, pd.DataFrame(table.records)])
        count += 1
        
names = ['veh_aux.csv', 'per_aux.csv', 'acc_aux.csv']

In [8]:
for name in log_progress.log_progress(names):
    for k in log_progress.log_progress(['f_2015', 'f_2016', 'f_2017']):
        if name == 'veh_aux.csv':
            df = pd.read_csv('%s/%s' % (k, name))
            df = df[VEH_AUX.columns]
            VEH_AUX = pd.concat([VEH_AUX, df])
        elif name == 'per_aux.csv':
            df = pd.read_csv('%s/%s' % (k, name))
            if k == 'f_2015':
                PER_AUX['A_RESTUSE'] = np.nan
                PER_AUX['A_HELMUSE'] = np.nan
            df['A_REST'] = np.nan
            df = df[PER_AUX.columns]
            PER_AUX = pd.concat([PER_AUX, df])
        elif name == 'acc_aux.csv':
            df = pd.read_csv('%s/%s' % (k, name))
            df = df[ACC_AUX.columns]
            ACC_AUX = pd.concat([ACC_AUX, df])


In [9]:
print("MBs: %.2f, %.2f, %.2f" % (VEH_AUX.memory_usage(deep=True).sum()/1024**2, PER_AUX.memory_usage(deep=True).sum()/1024**2, ACC_AUX.memory_usage(deep=True).sum()/1024**2))

MBs: 256.17, 657.65, 421.52


In [10]:
dataset = 'nrippner/fatality-analysis-reporting-system-fars-data-tables'
with ddw.open_remote_file(dataset, 'VEH_AUX.csv', mode='w') as f:
    VEH_AUX.to_csv(f, index=False)
with ddw.open_remote_file(dataset, 'PER_AUX.csv', mode='w') as f:
    PER_AUX.to_csv(f, index=False)
with ddw.open_remote_file(dataset, 'ACC_AUX.csv', mode='w') as f:
    ACC_AUX.to_csv(f, index=False)